# PRACTICE: NSMC DATASET

## Install Packages

In [2]:
!pip install portalocker
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.6 MB/s eta 0:00:00


## Dependency & Fix Seed

In [3]:
from torchtext.datasets import IMDB
import portalocker
import torchtext
import torch 
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torchtext.__version__

'0.15.1+cpu'

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from konlpy.tag import Okt  #You need to use customed tokenizer or Morphology Tagger to tokenize Original Sentence(Words). If you can't, Spacing could be the alternatives, but I don't recommend that much 


In [8]:
from torch.utils.data import Dataset

class MapDataset(Dataset):

    def __init__(self, data):
        self.data = data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return (self.data.iloc[index]['label'], self.data.iloc[index]['document']) # return label, document

## Load Dataset

In [13]:
import pandas as pd 
file_name = ''   # --> fill it
df = pd.read_table(file_name) # .dropna().reset_index().drop(columns = 'index')   --> Use Annotated Option if needed

In [14]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter = MapDataset(df)   # --> Use Map Style Dataset Which Contains __getitem__ protocal (conpatible with collate_fn)  
train_dataset = to_map_style_dataset()  # --> fill it

In [33]:
num_train = int(len(train_dataset) * 0.95)
train_dataset, valid_dataset = random_split()  # --> fill it, you need to split train and valid dataset with ur own ratio. 


# Tokenize & Build Vocab (Can Use Pretrained Embedding if you want following the way in LSTM notebook.

In [34]:
tokenizer = Okt()
def yield_tokens(data_iter):

    for (_, _text) in data_iter:
        yield tokenizer. # --> fill it, you need to know how to use Morphology Tagger(if you use) to tokenize raw sentence. Required Output type: list(str)
s 
vocab = build_vocab_from_iterator(yield_tokens(train_iter), 
                                  specials=[],   # --> fill it, which special token would you declare?
                                  max_tokens= )  # --> fill it, how many tokens in vocab?..... depends on you
vocab.set_default_index(vocab["<unk>"])

In [35]:
len(vocab)

25000

In [36]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

## Define collate_fn, you can customize it!

In [37]:
text_pipeline = lambda x: vocab()    # -->fill it, vocab(_____), same as above in yield_tokens
label_pipeline = lambda x: int(x) - 1


def custom_collate_fn(batch):  #--> Just Understand how it works
    label_list, text_list= [], []
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         
    label_list = torch.tensor(label_list, dtype=torch.int64).unsqueeze(dim=1)
    text_list = pad_sequence(text_list, padding_value = 1)
    return label_list.to(device), text_list.to(device)


## Create DataLoader Instance

In [38]:
train_dataloader = DataLoader() # -->fill it

valid_dataloader = DataLoader() # -->fill it

## Model (Use Model that you want!  vanilla RNN may not work well, though :))

In [39]:
import torch.nn as nn

#Use Any Deep-Model you want!, FFNN, RNN, CNN, ...
class MODEL(nn.Module):
    def __init__(self, ):
        
        super().__init__()
      
        
    def forward(self, text):
        
        return 

# Create Model Instance

In [40]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM =  #   -->fill it
HIDDEN_DIM =     #   -->fill it
OUTPUT_DIM =     #   -->fill it

model =MODEL()

## Num. of Parameters

In [41]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,591,905 trainable parameters


## The thing we need for Gradient-based Training! 

In [42]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

## Trace Accuracy per Epoch

In [43]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc * 100

# Train & Valid

In [44]:
import time


def train(dataloader):

    # -->fill it
    return 



def evaluate(dataloader):

   # -->fill it
    return 

In [45]:

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## RUN

In [46]:
EPOCHS =  # -->fill it

best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    start_time = time.time()

    train_loss, train_acc = train(train_dataloader)
    valid_loss, valid_acc = evaluate(valid_dataloader)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: ignored

## Further ..

In [ ]:
def inference():
  """
  DO YOURSELF!
  """